In [1]:
from pulp import *

In [2]:
listSolvers(onlyAvailable=True)

/home/tgellert/private_repositories/pulp/pulp/apis/gurobi_api.py:297: UserWarning: GUROBI error: .
  warnings.warn('GUROBI error: {}.'.format(out))


['GLPK_CMD', 'PULP_CBC_CMD', 'COIN_CMD', 'PULP_CHOCO_CMD', 'SCIP_CMD']

In [3]:
import random 

def create_bin_packin_problem(bins=4, seed=0):
    packed_bins = [[] for _ in range(bins)]
    bin_size = bins * 100
    random.seed(seed)
    for i in range(len(packed_bins)):
        remaining_size = bin_size
        while remaining_size >= 1:
            item = random.randrange(1, remaining_size + 10)
            packed_bins[i].append(item)
            remaining_size -= item
        packed_bins[i][-1] += remaining_size
    all_items_with_bin = [(n, i) for i, l in enumerate(packed_bins) for n in l]

    random.shuffle(all_items_with_bin)
    items, packing = zip(*all_items_with_bin)
    return items, packing, bin_size


In [4]:
from itertools import product

def create_problem(bins=15):
    items, packing, bin_size = create_bin_packin_problem(bins=bins)
    
    prob = LpProblem("bin_packing", LpMinimize)
    
    bin_indices = [i for i in range(len(items))]
    item_indices = [i for i in range(len(items))]

    using_bin = LpVariable.dicts("y", bin_indices, cat=LpBinary)
    items_packed = LpVariable.dicts("x", indexs=product(item_indices,bin_indices), cat=LpBinary)

    prob +=  lpSum(using_bin), "obj"

    
    #pack every item
    for i in item_indices:
        prob += lpSum(items_packed[i, b] for b in bin_indices) == 1, f"pack_item_{i}"
    
    # no bin overfilled
    for b in bin_indices:
        expr = lpSum([items[i] * items_packed[i, b] for i in item_indices]) <= bin_size * using_bin[b]
        prob += expr, f"respect_binm_sizue_{b}"
    
    return prob

In [5]:
bins = 10

In [6]:
# SCIP finds an optimal solution using 10 bins
prob = create_problem(bins)
solver = SCIP()
status = prob.solve(solver)

print(f"status: {LpStatus[status]}")
print(f"objective: {value(prob.objective):,.2f}")
print(f"time: {solver.solution_time:.2f}s")

status: Optimal
objective: 10.00
time: 1.75s


In [7]:
# SCIP finds an optimal solution using 11 bins
# absolute Gap: 1.5 → 1 bin more considered optimal

prob = create_problem(bins)
solver = SCIP(gapAbs=1.5)
status = prob.solve(solver)

print(f"status: {LpStatus[status]}")
print(f"objective: {value(prob.objective):,.2f}")
print(f"time: {solver.solution_time:.2f}s")

status: Optimal
objective: 11.00
time: 0.67s


In [8]:
# SCIP finds an optimal solution using 11 bins
# absolute Gap: 15% → 11 bins considered optimal

prob = create_problem(bins)
solver = SCIP(gapRel=0.15)
status = prob.solve(solver)
solver.getOptions()

print(f"status: {LpStatus[status]}")
print(f"objective: {value(prob.objective):,.2f}")
print(f"time: {solver.solution_time:.2f}s")

status: Optimal
objective: 11.00
time: 0.71s
